Plot Ross Ice Shelf Map
=======================

Demonstrates plotting hourly tidal displacements for the Ross Ice Shelf

OTIS format tidal solutions provided by Ohio State University and ESR  
- http://volkov.oce.orst.edu/tides/region.html  
- https://www.esr.org/research/polar-tide-models/list-of-polar-tide-models/
- ftp://ftp.esr.org/pub/datasets/tmd/  

Global Tide Model (GOT) solutions provided by Richard Ray at GSFC  

Finite Element Solution (FES) provided by AVISO  
- https://www.aviso.altimetry.fr/en/data/products/auxiliary-products/global-tide-fes.html

#### Python Dependencies
 - [numpy: Scientific Computing Tools For Python](https://www.numpy.org)  
 - [scipy: Scientific Tools for Python](https://www.scipy.org/)  
 - [pyproj: Python interface to PROJ library](https://pypi.org/project/pyproj/)  
 - [netCDF4: Python interface to the netCDF C library](https://unidata.github.io/netcdf4-python/)  
 - [matplotlib: Python 2D plotting library](http://matplotlib.org/)  
 - [cartopy: Python package designed for geospatial data processing](https://scitools.org.uk/cartopy/docs/latest/)  

#### Program Dependencies

- `calc_astrol_longitudes.py`: computes the basic astronomical mean longitudes  
- `calc_delta_time.py`: calculates difference between universal and dynamic time  
- `convert_ll_xy.py`: convert lat/lon points to and from projected coordinates  
- `load_constituent.py`: loads parameters for a given tidal constituent  
- `load_nodal_corrections.py`: load the nodal corrections for tidal constituents  
- `infer_minor_corrections.py`: return corrections for minor constituents  
- `model.py`: retrieves tide model parameters for named tide models  
- `read_tide_model.py`: extract tidal harmonic constants from OTIS tide models  
- `read_netcdf_model.py`: extract tidal harmonic constants from netcdf models  
- `read_GOT_model.py`: extract tidal harmonic constants from GSFC GOT models  
- `read_FES_model.py`: extract tidal harmonic constants from FES tide models  
- `predict_tide.py`: predict tidal elevation at a single time using harmonic constants  

This notebook uses Jupyter widgets to set parameters for calculating the tidal maps.  
The widgets can be installed as described below.  
```
pip3 install --user ipywidgets
jupyter nbextension install --user --py widgetsnbextension
jupyter nbextension enable --user --py widgetsnbextension
jupyter-notebook
```

#### Load modules

In [ ]:
import os
import pyproj
import datetime
import numpy as np
import matplotlib
matplotlib.rcParams['axes.linewidth'] = 2.0
matplotlib.rcParams["animation.html"] = "jshtml"
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import ipywidgets as widgets
from IPython.display import HTML

import pyTMD.time
import pyTMD.model
from pyTMD.calc_delta_time import calc_delta_time
from pyTMD.read_tide_model import extract_tidal_constants
from pyTMD.read_netcdf_model import extract_netcdf_constants
from pyTMD.read_GOT_model import extract_GOT_constants
from pyTMD.read_FES_model import extract_FES_constants
from pyTMD.infer_minor_corrections import infer_minor_corrections
from pyTMD.predict_tide import predict_tide
# autoreload
%load_ext autoreload
%autoreload 2

#### Set parameters  for program

- Model directory  
- Tide model  
- Date to run  

In [ ]:
# set the directory with tide models
dirText = widgets.Text(
    value=os.getcwd(),
    description='Directory:',
    disabled=False
)

# dropdown menu for setting tide model
model_list = sorted(pyTMD.model.global_ocean() + pyTMD.model.antarctic_ocean())
modelDropdown = widgets.Dropdown(
    options=model_list,
    value='CATS2008',
    description='Model:',
    disabled=False,
)

# dropdown menu for setting ATLAS format model
atlas_list = ['OTIS','netcdf']
atlasDropdown = widgets.Dropdown(
    options=atlas_list,
    value='netcdf',
    description='ATLAS:',
    disabled=False,
)

# checkbox for setting if tide files are compressed
compressCheckBox = widgets.Checkbox(
    value=True,
    description='Compressed?',
    disabled=False,
)

# date picker widget for setting time
datepick = widgets.DatePicker(
    description='Date:',
    value = datetime.date.today(),
    disabled=False
)

# display widgets for setting directory and model
widgets.VBox([
    dirText,
    modelDropdown,
    atlasDropdown,
    compressCheckBox,
    datepick
])

#### Setup tide model parameters

In [ ]:
# get model parameters
model = pyTMD.model(dirText.value, format=atlasDropdown.value,
    compressed=compressCheckBox.value).elevation(modelDropdown.value)

#### Setup coordinates for calculating tides

In [ ]:
# create an image around the Ross Ice Shelf
xlimits = [-740000,520000]
ylimits = [-1430000,-300000]
spacing = [5e3,-5e3]
# x and y coordinates
x = np.arange(xlimits[0],xlimits[1]+spacing[0],spacing[0])
y = np.arange(ylimits[1],ylimits[0]+spacing[1],spacing[1])
xgrid,ygrid = np.meshgrid(x,y)
# x and y dimensions
nx = int((xlimits[1]-xlimits[0])/spacing[0])+1
ny = int((ylimits[0]-ylimits[1])/spacing[1])+1
# convert image coordinates from polar stereographic to latitude/longitude
crs1 = pyproj.CRS.from_string("epsg:{0:d}".format(3031))
crs2 = pyproj.CRS.from_string("epsg:{0:d}".format(4326))
transformer = pyproj.Transformer.from_crs(crs1, crs2, always_xy=True)
lon,lat = transformer.transform(xgrid.flatten(), ygrid.flatten())

#### Calculate tide map

In [ ]:
# convert from calendar date to days relative to Jan 1, 1992 (48622 MJD)
YMD = datepick.value
tide_time = pyTMD.time.convert_calendar_dates(YMD.year, YMD.month,
    YMD.day, hour=np.arange(24))
# delta time (TT - UT1) file
delta_file = pyTMD.utilities.get_data_path(['data','merged_deltat.data'])

# read tidal constants and interpolate to grid points
if model.format in ('OTIS','ATLAS'):
    amp,ph,D,c = extract_tidal_constants(lon, lat, model.grid_file,
        model.model_file, model.projection, TYPE=model.type,
        METHOD='spline', GRID=model.format)
    DELTAT = np.zeros_like(tide_time)
elif (model.format == 'netcdf'):
    amp,ph,D,c = extract_netcdf_constants(lon, lat, model.grid_file,
        model.model_file, TYPE=model.type, METHOD='spline',
        SCALE=model.scale, GZIP=model.compressed)
    DELTAT = np.zeros_like(tide_time)
elif (model.format == 'GOT'):
    amp,ph,c = extract_GOT_constants(lon, lat, model.model_file,
        METHOD='spline', SCALE=model.scale, GZIP=model.compressed)
    # interpolate delta times from calendar dates to tide time
    DELTAT = calc_delta_time(delta_file, tide_time)
elif (model.format == 'FES'):
    amp,ph = extract_FES_constants(lon, lat, model.model_file,
        TYPE=model.type, VERSION=model.version, METHOD='spline',
        SCALE=model.scale, GZIP=model.compressed)
    c = model.constituents
    # interpolate delta times from calendar dates to tide time
    DELTAT = calc_delta_time(delta_file, tide_time)
    
# calculate complex phase in radians for Euler's
cph = -1j*ph*np.pi/180.0
# calculate constituent oscillation
hc = amp*np.exp(cph)
    
# allocate for tide map calculated every hour
tide_cm = np.ma.zeros((ny,nx,24))
for hour in range(24):
    # predict tidal elevations at time and infer minor corrections
    TIDE = predict_tide(tide_time[hour], hc, c, DELTAT=DELTAT[hour],
        CORRECTIONS=model.format)
    MINOR = infer_minor_corrections(tide_time[hour], hc, c,
        DELTAT=DELTAT[hour], CORRECTIONS=model.format)
    # add major and minor components and reform grid
    # convert from meters to centimeters
    tide_cm[:,:,hour] = 100.0*np.reshape((TIDE+MINOR),(ny,nx))

#### Create animation of hourly tidal oscillation

In [ ]:
# output Ross Ice Shelf Tide Animation
projection = ccrs.Stereographic(central_longitude=0.0,
    central_latitude=-90.0,true_scale_latitude=-71.0)
fig, ax = plt.subplots(num=1, figsize=(9,8),
    subplot_kw=dict(projection=projection))
# plot tide height
vmin,vmax = (np.min(tide_cm), np.max(tide_cm))
extent = (xlimits[0],xlimits[1],ylimits[0],ylimits[1])
im = ax.imshow(np.zeros((ny,nx)), interpolation='nearest',
    vmin=vmin, vmax=vmax, transform=projection,
    extent=extent, origin='upper', animated=True)
# add high resolution cartopy coastlines
ax.coastlines('10m')

# Add colorbar and adjust size
# pad = distance from main plot axis
# extend = add extension triangles to upper and lower bounds
# options: neither, both, min, max
# shrink = percent size of colorbar
# aspect = lengthXwidth aspect of colorbar
cbar = plt.colorbar(im, ax=ax, pad=0.025, extend='both',
    extendfrac=0.0375, shrink=0.85, aspect=22.5, drawedges=False)
# rasterized colorbar to remove lines
cbar.solids.set_rasterized(True)
# Add label to the colorbar
cbar.ax.set_ylabel('{0} Tide Height'.format(model.name), fontsize=13)
cbar.ax.set_xlabel('cm', fontsize=13)
cbar.ax.xaxis.set_label_coords(0.50, 1.04)
# ticks lines all the way across
cbar.ax.tick_params(which='both', width=1, length=18,
    labelsize=13, direction='in')
# add title (date and time)
ttl = ax.set_title(None, fontsize=13)
# set x and y limits
ax.set_xlim(xlimits)
ax.set_ylim(ylimits)

# stronger linewidth on frame
ax.spines['geo'].set_linewidth(2.0)
ax.spines['geo'].set_capstyle('projecting')
# adjust subplot within figure
fig.subplots_adjust(left=0.02,right=0.98,bottom=0.05,top=0.98)
           
# animate each map
def animate_maps(hour):
    # set map data
    im.set_data(tide_cm[:,:,hour])
    # set title
    args = (YMD.year,YMD.month,YMD.day,hour)
    ttl.set_text('{0:4d}-{1:02d}-{2:02d}T{3:02d}:00:00'.format(*args))

# set animation
anim = animation.FuncAnimation(fig, animate_maps, frames=24)
%matplotlib inline
HTML(anim.to_jshtml())